___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Keras RL DQN on Image Environment - Exercise - 




In thise notebook you will implement a DQN agent on the famous game of Pong:
**Use the Pong-v0 environment**
(https://gym.openai.com/envs/Pong-v0/) <br />

**TASK: Import necessary libraries and create the environment. Also extract the possible actions** <br />

In [1]:
from PIL import Image
import numpy as np
import gym

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam

In [3]:
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [4]:
from gym.utils import play

pygame 2.0.1 (SDL 2.0.14, Python 3.6.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
env = gym.make("Pong-v0")

In [5]:
nb_actions = env.action_space.n

In [6]:
nb_actions

6

**TASK: Play the game manually (keys: a and d to move the bars)** <br />

In [ ]:
play.play(env)

**TASK: Define an input size and the window length** <br />

In [7]:
IMG_SHAPE = (84, 84)
WINDOW_LENGTH = 4

**TASK: Create the ImageProcessor** <br />
It needs to:
1. Resize the image
2. Convert it to grayscale
3. Standardize it
4. Be memory efficient

Dont forget the reward clipping

In [8]:
class ImageProcessor(Processor):
    def process_observation(self, observation):
        img = Image.fromarray(observation)
        img = img.resize(IMG_SHAPE)
        img = img.convert("L")
        img = np.array(img)        
        return img.astype("uint8")
    
    def process_state_batch(self, batch):
        process_batch = batch.astype("float32") / 255.0
        return process_batch
    
    def process_reward(self, reward):
        return np.clip(reward, -1.0, 1.0)

In [9]:
input_shape = (WINDOW_LENGTH, IMG_SHAPE[0], IMG_SHAPE[1])
input_shape

(4, 84, 84)

**TASK: Design the Convolutional Neural Network** <br />
Hint: Make sure to get the right input shape!

You can try the same architecture than presented in the previous notebook:
1. Conv2D(filters=32, kernel_size=8, stride=4)
2. Conv2D(filters=64, kernel_size=4, stride=2)
3. Conv2D(filters=64, kernel_size=3, stride=1)
4. Dense(512)

Dont forget the activation function

In [10]:
model = Sequential()

In [11]:
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4), kernel_initializer="he_normal"))
model.add(Activation("relu"))
model.add(Convolution2D(64, (4, 4), strides=(2, 2), kernel_initializer="he_normal"))
model.add(Activation("relu"))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), kernel_initializer="he_normal"))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(nb_actions))
model.add(Activation("linear"))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
activation (Activation)      (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_1 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 64)          0

**TASK: Create the Replay Memory** <br />


In [13]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

**TASK: Create the processor** <br />


In [14]:
processor = ImageProcessor()

**TASK: Define the action selection policy.** <br />
Feel free to try all policies you like. (Hint: decaying epsilon greedy also works here)

In [15]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr="eps",
                              value_max=1.0,
                              value_min=0.1,
                              value_test=0.05,
                              nb_steps=1000000
                             )

**TASK: Create the agent.** <br />
Dont forget to compile!

In [16]:
dqn = DQNAgent(model=model,
               nb_actions=nb_actions,
               policy=policy,
               memory=memory,
               processor=processor,
               nb_steps_warmup=50000,
               gamma=0.99,
               target_model_update=10000,
               delta_clip=1
              )

In [17]:
dqn.compile(Adam(learning_rate=0.00025), metrics=["mae"])

**TASK: Define a checkpoint callback to store the weights during training.** <br />
Please name it differently than our provided checkpoint to avoid overwriting it

In [18]:
weights_filename = "DQN_BO.h5f"
checkpoint_filename = "DQN_CHECKPOINT.h5f"

In [19]:
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_filename,
                                              interval=100000
                                             )

**TASK: Train the agent.** <br />

In [20]:
dqn.fit(env, nb_steps=1500000, callbacks=[checkpoint_callback], log_interval=500, visualize=False)

Training for 1500000 steps ...
Interval 1 (0 steps performed)


C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


500/500 [==============================] - 11s 6ms/step - reward: -0.0180
Interval 2 (500 steps performed)
500/500 [==============================] - 3s 6ms/step - reward: -0.0220
Interval 3 (1000 steps performed)
500/500 [==============================] - 3s 6ms/step - reward: -0.0120
1 episodes - episode_reward: -21.000 [-21.000, -21.000] - ale.lives: 0.000

Interval 4 (1500 steps performed)
500/500 [==============================] - 3s 6ms/step - reward: -0.0220
Interval 5 (2000 steps performed)
500/500 [==============================] - 3s 6ms/step - reward: -0.0180
1 episodes - episode_reward: -20.000 [-20.000, -20.000] - ale.lives: 0.000

Interval 6 (2500 steps performed)
500/500 [==============================] - 3s 7ms/step - reward: -0.0060
Interval 7 (3000 steps performed)
500/500 [==============================] - 3s 7ms/step - reward: -0.0160
Interval 8 (3500 steps performed)
500/500 [==============================] - 3s 6ms/step - reward: -0.0200
1 episodes - episode_rewar

500/500 [==============================] - 3s 7ms/step - reward: -0.0120
Interval 61 (30000 steps performed)
500/500 [==============================] - 3s 7ms/step - reward: -0.0200
1 episodes - episode_reward: -18.000 [-18.000, -18.000] - ale.lives: 0.000

Interval 62 (30500 steps performed)
500/500 [==============================] - 3s 7ms/step - reward: -0.0180
Interval 63 (31000 steps performed)
500/500 [==============================] - 3s 7ms/step - reward: -0.0100
1 episodes - episode_reward: -20.000 [-20.000, -20.000] - ale.lives: 0.000

Interval 64 (31500 steps performed)
500/500 [==============================] - 3s 7ms/step - reward: -0.0220
Interval 65 (32000 steps performed)
500/500 [==============================] - 3s 7ms/step - reward: -0.0100
Interval 66 (32500 steps performed)
500/500 [==============================] - 3s 7ms/step - reward: -0.0080
1 episodes - episode_reward: -20.000 [-20.000, -20.000] - ale.lives: 0.000

Interval 67 (33000 steps performed)
500/500 [

500/500 [==============================] - 43s 86ms/step - reward: -0.0200
Interval 116 (57500 steps performed)
500/500 [==============================] - 44s 87ms/step - reward: -0.0220
1 episodes - episode_reward: -21.000 [-21.000, -21.000] - loss: 0.006 - mae: 0.406 - mean_q: 0.494 - mean_eps: 0.948 - ale.lives: 0.000

Interval 117 (58000 steps performed)
500/500 [==============================] - 49s 98ms/step - reward: -0.0200
Interval 118 (58500 steps performed)
500/500 [==============================] - 47s 94ms/step - reward: -0.0200TA: 0s
Interval 119 (59000 steps performed)
500/500 [==============================] - 49s 99ms/step - reward: -0.0200
1 episodes - episode_reward: -21.000 [-21.000, -21.000] - loss: 0.005 - mae: 0.407 - mean_q: 0.491 - mean_eps: 0.947 - ale.lives: 0.000

Interval 120 (59500 steps performed)
500/500 [==============================] - 45s 89ms/step - reward: -0.0220
Interval 121 (60000 steps performed)
500/500 [==============================] - 43s 8

...

500/500 [==============================] - 63s 126ms/step - reward: -0.0020
Interval 2731 (1365000 steps performed)
500/500 [==============================] - 63s 126ms/step - reward: 0.0000e+00
Interval 2732 (1365500 steps performed)
500/500 [==============================] - 63s 126ms/step - reward: -0.0040
1 episodes - episode_reward: -10.000 [-10.000, -10.000] - loss: 0.006 - mae: 1.347 - mean_q: 1.636 - mean_eps: 0.100 - ale.lives: 0.000

Interval 2733 (1366000 steps performed)
500/500 [==============================] - 63s 126ms/step - reward: -0.0060
Interval 2734 (1366500 steps performed)
500/500 [==============================] - 63s 126ms/step - reward: -0.0120
Interval 2735 (1367000 steps performed)
500/500 [==============================] - 64s 128ms/step - reward: -0.0060
Interval 2736 (1367500 steps performed)
500/500 [==============================] - 63s 126ms/step - reward: -0.0040
Interval 2737 (1368000 steps performed)
500/500 [==============================] - 64s 1

500/500 [==============================] - 67s 135ms/step - reward: 0.0000e+00
Interval 2789 (1394000 steps performed)
500/500 [==============================] - 68s 135ms/step - reward: -0.0120
Interval 2790 (1394500 steps performed)
500/500 [==============================] - 67s 135ms/step - reward: -0.0080
Interval 2791 (1395000 steps performed)
500/500 [==============================] - 68s 135ms/step - reward: -0.0040
Interval 2792 (1395500 steps performed)
500/500 [==============================] - 68s 135ms/step - reward: -0.0120
1 episodes - episode_reward: -17.000 [-17.000, -17.000] - loss: 0.007 - mae: 1.342 - mean_q: 1.630 - mean_eps: 0.100 - ale.lives: 0.000

Interval 2793 (1396000 steps performed)
500/500 [==============================] - 68s 135ms/step - reward: 0.0000e+00
Interval 2794 (1396500 steps performed)
500/500 [==============================] - 67s 135ms/step - reward: -0.0060
Interval 2795 (1397000 steps performed)
500/500 [==============================] - 68

500/500 [==============================] - 67s 135ms/step - reward: -0.0040
Interval 2847 (1423000 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: -0.0060
Interval 2848 (1423500 steps performed)
500/500 [==============================] - 67s 135ms/step - reward: -0.0060
Interval 2849 (1424000 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: 0.0000e+00
Interval 2850 (1424500 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: 0.0000e+00
Interval 2851 (1425000 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: -0.0100
1 episodes - episode_reward: -13.000 [-13.000, -13.000] - loss: 0.007 - mae: 1.377 - mean_q: 1.671 - mean_eps: 0.100 - ale.lives: 0.000

Interval 2852 (1425500 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: -0.0060
Interval 2853 (1426000 steps performed)
500/500 [==============================] - 67

500/500 [==============================] - 68s 135ms/step - reward: -0.0120
Interval 2904 (1451500 steps performed)
500/500 [==============================] - 68s 135ms/step - reward: -0.0040
Interval 2905 (1452000 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: -0.0040
Interval 2906 (1452500 steps performed)
500/500 [==============================] - 67s 135ms/step - reward: -0.0060
1 episodes - episode_reward: -16.000 [-16.000, -16.000] - loss: 0.008 - mae: 1.383 - mean_q: 1.679 - mean_eps: 0.100 - ale.lives: 0.000

Interval 2907 (1453000 steps performed)
500/500 [==============================] - 68s 135ms/step - reward: -0.0040
Interval 2908 (1453500 steps performed)
500/500 [==============================] - 68s 136ms/step - reward: 0.0040
Interval 2909 (1454000 steps performed)
500/500 [==============================] - 68s 136ms/step - reward: -0.0080
Interval 2910 (1454500 steps performed)
500/500 [==============================] - 68s 135ms

500/500 [==============================] - 79s 158ms/step - reward: -0.0080
1 episodes - episode_reward: -11.000 [-11.000, -11.000] - loss: 0.007 - mae: 1.384 - mean_q: 1.681 - mean_eps: 0.100 - ale.lives: 0.000

Interval 2963 (1481000 steps performed)
500/500 [==============================] - 124s 248ms/step - reward: -0.0020
Interval 2964 (1481500 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: -0.0120
Interval 2965 (1482000 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: 0.0000e+00
Interval 2966 (1482500 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: -0.0020
Interval 2967 (1483000 steps performed)
500/500 [==============================] - 67s 134ms/step - reward: -0.0080
1 episodes - episode_reward: -15.000 [-15.000, -15.000] - loss: 0.007 - mae: 1.383 - mean_q: 1.679 - mean_eps: 0.100 - ale.lives: 0.000

Interval 2968 (1483500 steps performed)
500/500 [=================

**TASK: Evaluate the agent.** <br />

In [21]:
dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
Episode 1: reward: -12.000, steps: 3886


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [

**TASK: Load your weights (or the provided ones) and create an agent from those** <br />

In [ ]:
#TODO